In [1]:
 import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
from pathlib import Path
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from lockdowndates.core import LockdownDates

from catboost import CatBoostRegressor
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
from datetime import timedelta, date

In [2]:
X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")

In [3]:
X_train, y_train, X_test, y_test = utils.train_test_split_temporal(X, y, delta_threshold="30 days")

## Dense

In [4]:
num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id']
time_features = ['hour','month','weekday','day']



col_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(sparse=False), cat_features),
        #('time', FunctionTransformer(utils.encode_cyclical_features), time_features)
    ],
    remainder='passthrough'
)
X_train_transformed = col_transformer.fit_transform(utils.prepro(X_train))
X_test_transformed = col_transformer.fit_transform(utils.prepro(X_test))

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_transformed.shape[1],)))
#model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))

#model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(1))  # Pour la régression

model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(X_train_transformed, y_train, epochs=250, batch_size=64, callbacks=[early_stopping], validation_data=(X_test_transformed, y_test))

mse = model.evaluate(X_test_transformed, y_test)
# Évaluation du modèle
model.evaluate(X_test_transformed, y_test)

Epoch 1/250
7133/7133 [==============================] - 7s 906us/step - loss: 1.4543 - val_loss: 0.8779
Epoch 2/250
7133/7133 [==============================] - 7s 935us/step - loss: 0.6421 - val_loss: 0.5843
Epoch 3/250
7133/7133 [==============================] - 7s 923us/step - loss: 0.5703 - val_loss: 0.5124
Epoch 4/250
7133/7133 [==============================] - 7s 918us/step - loss: 0.5369 - val_loss: 0.4894
Epoch 5/250
7133/7133 [==============================] - 7s 931us/step - loss: 0.5044 - val_loss: 0.4730
Epoch 6/250
7133/7133 [==============================] - 6s 906us/step - loss: 0.4750 - val_loss: 0.4862
Epoch 7/250
7133/7133 [==============================] - 6s 904us/step - loss: 0.4542 - val_loss: 0.5660
Epoch 8/250
7133/7133 [==============================] - 6s 901us/step - loss: 0.4303 - val_loss: 0.4882
Epoch 9/250
7133/7133 [==============================] - 6s 900us/step - loss: 0.4172 - val_loss: 0.5743
Epoch 10/250
7133/7133 [==============================]

0.2959691882133484

In [61]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')

3106/3106 [==============================] - 1s 261us/step
Root Mean Squared Error: 0.49068946558443444


In [62]:
X_final_test_scaled = col_transformer.fit_transform(utils.prepro(X_final_test))

submission = model.predict(X_final_test_scaled)
pd.DataFrame(submission, columns=["log_bike_count"]).rename_axis('Id').to_csv('submission26_deep_101223.csv')

1608/1608 [==============================] - 0s 295us/step


In [6]:
X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")

In [ ]:
num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id']
time_features = ['hour','month','weekday','day']



col_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(sparse=False), cat_features),
        #('time', FunctionTransformer(utils.encode_cyclical_features), time_features)
    ],
    remainder='passthrough'
)


In [ ]:
# Création des séquences
sequence_length = 24  # 24 heures de données par séquence
num_sequences_per_counter = 3888 // sequence_length

sequences = []
targets = []

for counter_id in X['counter_id'].unique():
    counter_index = X[X['counter_id'] == counter_id].index
    counter_data = X_scaled[counter_index]
    counter_target = y[counter_index]

    for i in range(num_sequences_per_counter):
        start_index = i * sequence_length
        end_index = start_index + sequence_length

        if end_index <= counter_data.shape[0]:
            sequence = counter_data[start_index:end_index]
            target = counter_target.iloc[end_index - 1]

            sequences.append(sequence)
            targets.append(target)

X_sequences = np.array(sequences)
y_targets = np.array(targets)



In [23]:
num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id']
time_features = ['hour','month','weekday','day']

col_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(sparse=False), cat_features),
        ('time', FunctionTransformer(utils.encode_cyclical_features), time_features)
    ],
    remainder='passthrough'
)

X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
cat = CatBoostRegressor(
    depth=10,
    iterations=1000,
    rsm=0.25,
    sampling_frequency="PerTree",
    subsample=0.7,
    verbose=0,
)

pipe = Pipeline([
    ('prepro',FunctionTransformer(utils.prepro)),
    ('col', col_transformer),
    ('model', cat)
])


pipe.fit(X_train, y_train)

predictions = pipe.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 0.3468958394351414


In [25]:
catboost_predictions_train = pipe.predict(X_train)
catboost_predictions_test = pipe.predict(X_test)

In [26]:
pipe2 = Pipeline([
    ('prepro',FunctionTransformer(utils.prepro)),
    ('col', col_transformer),
])
X_train_prepro = pipe2.fit_transform(X_train)
X_test_prepro = pipe2.fit_transform(X_test)

In [ ]:
X_train_with_predictions = np.hstack((X_train_prepro, catboost_predictions_train.reshape(-1, 1)))
X_test_with_predictions = np.hstack((X_test_prepro, catboost_predictions_test.reshape(-1, 1)))

# Création des séquences
sequence_length = 24  # 24 heures de données par séquence
num_sequences_per_counter = 3888 // sequence_length

sequences = []
targets = []

for counter_id in X['counter_id'].unique():
    counter_index = X[X['counter_id'] == counter_id].index
    counter_data = X_scaled[counter_index]
    counter_target = y[counter_index]

    for i in range(num_sequences_per_counter):
        start_index = i * sequence_length
        end_index = start_index + sequence_length

        if end_index <= counter_data.shape[0]:
            sequence = counter_data[start_index:end_index]
            target = counter_target.iloc[end_index - 1]

            sequences.append(sequence)
            targets.append(target)

X_sequences = np.array(sequences)
y_targets = np.array(targets)



In [7]:
X_train, y_train, X_test, y_test = utils.train_test_split_temporal(X, y, delta_threshold="30 days")
X_train_transformed = col_transformer.fit_transform(utils.prepro(X_train))
X_test_transformed = col_transformer.fit_transform(utils.prepro(X_test))

In [14]:
X_sequences = np.array([X_train_transformed[i:i+sequence_length] for i in range(len(X_train_transformed)-sequence_length)])
X_test_sequences = np.array([X_test_transformed[i:i+sequence_length] for i in range(len(X_test_transformed)-sequence_length)])

y_sequences = np.array([y_train[i+sequence_length] for i in range(len(y_train)-sequence_length)])
y_test_sequences = np.array([y_test[i+sequence_length] for i in range(len(y_test)-sequence_length)])

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Taille des séquences, à définir en fonction de vos données
sequence_length = 24

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, X_sequences.shape[2])))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))  # Ajustez en fonction de votre tâche (régression ou classification)

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_sequences, y_sequences, epochs=20, batch_size=64, validation_data=(X_test_sequences, y_test_sequences))

Epoch 1/20
7133/7133 [==============================] - 52s 7ms/step - loss: 17.6811 - val_loss: 6.0416
Epoch 2/20
7133/7133 [==============================] - 49s 7ms/step - loss: 5.6824 - val_loss: 2.0361
Epoch 3/20
7133/7133 [==============================] - 49s 7ms/step - loss: 2.1582 - val_loss: 1.9593
Epoch 4/20
7133/7133 [==============================] - 49s 7ms/step - loss: 1.2578 - val_loss: 0.7439
Epoch 5/20
7133/7133 [==============================] - 49s 7ms/step - loss: 1.0291 - val_loss: 0.8140
Epoch 6/20
7133/7133 [==============================] - 50s 7ms/step - loss: 0.8238 - val_loss: 0.7993
Epoch 7/20
7133/7133 [==============================] - 51s 7ms/step - loss: 0.6769 - val_loss: 0.6311
Epoch 8/20
7133/7133 [==============================] - 50s 7ms/step - loss: 0.7257 - val_loss: 0.5631
Epoch 9/20
7133/7133 [==============================] - 45s 6ms/step - loss: 0.5896 - val_loss: 0.6606
Epoch 10/20
7133/7133 [==============================] - 44s 6ms/step - 

In [21]:
def predict_LSTM_X_test(X_test):
    X_test_transformed = col_transformer.fit_transform(utils.prepro(X_test))
    return model.predict(np.array([X_test_transformed[i:i+sequence_length] for i in range(len(X_test_transformed)-sequence_length)]))



pd.DataFrame(predict_LSTM_X_test(X_final_test), columns=["log_bike_count"]).rename_axis('Id')#.to_csv('submission30_LSTMTEST_101223.csv')

1607/1607 [==============================] - 2s 1ms/step


,log_bike_count
Id,
0,1.079689
1,3.295458
2,0.335061
3,0.410036
4,0.812615
...,...
51411,0.891897
51412,1.620866
51413,2.531311


In [22]:
X_final_test

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 01:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 13:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 17:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 19:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,2021-09-10 22:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429
...,...,...,...,...,...,...,...,...,...,...
51435,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,2021-10-18 11:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980
51436,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,2021-10-18 15:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980
51437,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,2021-10-18 17:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980
51438,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,2021-10-18 18:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980
